In [ ]:

%pprint
import sys
import os.path as osp
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)

# Import needed libraries
import subprocess
import os
import pyperclip

In [ ]:

def check_push_rights_to_repos(github_folder):
    """
    Checks if the current user has push rights to all repositories in a specified GitHub folder.

    Args:
        github_folder (str): The path to the folder containing Git repositories.

    Returns:
        dict: A dictionary where the keys are repository names and the values are booleans
              indicating whether the user has push rights (True) or not (False).
    """
    if not os.path.isdir(github_folder):
        raise ValueError("The specified path is not a valid directory.")
    
    push_rights = {}
    
    # List all subdirectories in the GitHub folder
    for repo_name in os.listdir(github_folder):
        repo_path = os.path.join(github_folder, repo_name)
        
        # Check if the subdirectory is a Git repository
        if os.path.isdir(repo_path) and os.path.isdir(os.path.join(repo_path, '.git')):
            try:
                # Use subprocess to check push access
                result = subprocess.run(
                    ["git", "-C", repo_path, "push", "--dry-run"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True
                )
                
                # If there's no error in the dry-run, the user has push access
                if result.returncode == 0:
                    push_rights[repo_name] = True
                else:
                    push_rights[repo_name] = False
            except Exception as e:
                # Handle unexpected errors during the subprocess call
                push_rights[repo_name] = False
        else:
            # If it's not a Git repository, skip it
            push_rights[repo_name] = False
    
    return push_rights

In [ ]:

# In RegexBuddy, place *.ipynb; in the "File masks:" field, for example, and paste this in the "Folders:" field
push_rights = check_push_rights_to_repos(os.path.dirname(nu.github_folder))
pyperclip.copy('"' + '", "'.join([
    f'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\{repo_folder}' for repo_folder, is_pushable in push_rights.items() if is_pushable
]) + '"')